In [ ]:
from ultralytics import YOLO 
import cv2
import os
from paddleocr import PaddleOCR
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import csv
import time

In [ ]:
def to_polar(img):
    """
    Преобразует исходное изображение, делая его развёртку 360 градусов по полярной системе координат.
    Далее возвращаем новое изображение, на котором будем распознавать текст
    """
    rows = img.shape[0]
    cols = img.shape[1]    
    max_ = max(rows, cols)
    img = cv2.resize(img, (max_, max_))    
    center = (int(max_/ 2), int(max_/ 2))
    radius = min(center[0], center[1])    
    polar = cv2.warpPolar(img, dsize=(2 * radius, 2 * radius), center=center, maxRadius=radius, flags=cv2.WARP_POLAR_LINEAR)
    polar = cv2.rotate(polar, cv2.ROTATE_90_COUNTERCLOCKWISE)    
    res = cv2.hconcat([polar, polar])
    return res

In [ ]:
def detect_and_recognition(img, model, circle_yolo, ocr):
    # Детектирование областей с текстом с помощью YOLO
    circle_res = circle_yolo.predict(img)
    result_yolo = model.predict(img)

    # Проверка на круглые детали для дальнейшей обработки
    is_circle = False
    if len(circle_res[0].boxes.cls):
        is_circle = True

    # Лист распознанного текста
    text = []

    # Распознавание текста в каждой области с помощью paddleocr
    for i in range(len(result_yolo)):
        
        boxes_yolo = result_yolo[i].boxes
        for j in range(len(boxes_yolo)):
            # Выделение каждой области с текстом 
            x1_yolo, y1_yolo, x2_yolo, y2_yolo = boxes_yolo.xyxy[j]
            x1_yolo, y1_yolo, x2_yolo, y2_yolo = int(x1_yolo), int(y1_yolo), int(x2_yolo), int(y2_yolo)
            cropped_image = img[y1_yolo:y2_yolo, x1_yolo:x2_yolo]

            # Если деталь круглая, то преобразуем изображение в полярную систему координат
            if is_circle:
                   cropped_image_new = to_polar(cropped_image)
            else: cropped_image_new = cropped_image

            result_ocr = ocr.ocr(cropped_image_new, cls=False) # Распознаём текст на изображении с помощью paddleocr

            if result_ocr[0] != None:

                # Визуализируем боксы на исходном изображении
                result_ocr = result_ocr[0]
                txts = [line[1][0] for line in result_ocr]
                text.append(txts)
                
                boxes_ocr = [line[0] for line in result_ocr]

                boxes_ocr = np.array(boxes_ocr)
                for i in range(len(boxes_ocr)):
                    
                    # Смещаем боксы от paddleocr к боксам, которые выделила yolo
                    boxes_ocr[i][:, 1] += y1_yolo
                    boxes_ocr[i][:, 0] += x1_yolo

                    p1 = (int(boxes_ocr[i][:, 0].max()), int(boxes_ocr[i][:, 1].max()))
                    p0 = (int(boxes_ocr[i][:, 0].min()), int(boxes_ocr[i][:, 1].min()))

                    cv2.rectangle(img, (int(boxes_ocr[i][:, 0].min()), int(boxes_ocr[i][:, 1].min())), (int(boxes_ocr[i][:, 0].max()), int(boxes_ocr[i][:, 1].max())), (255, 0, 0), 2)

            else:
                print("No text detected.") # Если текст не распознан, то выводим сообщение об этом
            
    return text, img, result_yolo

In [ ]:
path_yolo = 'yolo/detection/best.pt'
path_yolo_circle = 'yolo/circle/best.pt'

In [ ]:
# Загрузка необходимых моделей
model = YOLO(path_yolo)
model_circle = YOLO(path_yolo)
ocr_en = PaddleOCR(lang='en')

### Путь к тестовым изображениям

### Необходимо поменять путь к датасету

In [ ]:
PATH_TO_IMGS = "D:/VSC/Text/train Росатом/train/imgs"

In [ ]:
from difflib import SequenceMatcher
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import io
from itertools import combinations, chain
import pandas as pd


local_file_path = 'ДеталиПоПлануДляРазрешенныхЗаказов (2).xlsx'


def similarity(a, b):
    """Возвращает коэффициент схожести между двумя строками."""
    return SequenceMatcher(lambda x: x in " -_.\\/[]'\"", a, b, autojunk=True).ratio()


def find_best_match(df, search_string):
    max_similarity = 0
    best_row_index = -1

    for index, row in df.iterrows():
        combined_text = str(row[0]) + str(row[1])

        search_strings = ' '.join(search_string[0]) # Объединим все найденные строки в одну
        #print(search_strings)
        # Считаем схожесть
        avg_similarity = similarity(combined_text, search_strings)

        if avg_similarity > max_similarity:
            max_similarity = avg_similarity
            best_row_index = index

    return best_row_index


def find_best_match_all_combinations(table, snippets):
    if table.empty:
        return -1

    best_match_index = -1
    max_similarity = 0

    # Генерируем все возможные склейки отрывков
    for i in range(1, len(snippets) + 1):
        for snippet_combination in combinations(snippets, i):
            # Склеиваем отрывки в одну строку
            combined_snippet = " ".join(chain.from_iterable(snippet_combination))  # Склейка элементов списков
            snippet_vector = vectorizer.transform([combined_snippet])

            # Вычисляем косинусное сходство для каждой строки в таблице
            similarities = cosine_similarity(snippet_vector, table_vectors)

            # Находим строку с максимальным сходством
            current_best_index = np.argmax(similarities)
            current_best_similarity = similarities[0, current_best_index]

            # Обновляем наилучшее совпадение, если текущее сходство больше
            if current_best_similarity > max_similarity:
                best_match_index = current_best_index
                max_similarity = current_best_similarity

    return best_match_index

def find_in_BD(text, df):
    #best_match_index = find_best_match(df, text)
    best_match_index = find_best_match_all_combinations(df, text)

    best_match_row = df.iloc[best_match_index]  # Получаем всю строку по индексу
    print("Строка с максимальным совпадением:")
    print(best_match_row)

    return best_match_row


df = pd.read_excel(local_file_path)
# Объединяем первый и второй столбцы для каждой строки
combined_text = df.iloc[:, :2].apply(lambda row: ' '.join(row.astype(str)), axis=1)

# Используем TF-IDF для векторизации текста
vectorizer = TfidfVectorizer()
table_vectors = vectorizer.fit_transform(combined_text)

In [ ]:
import re

def remove_russian_chars(text):
    """
    Удаляет русские символы из строки.
    
    Параметры:
    text (str): Входная строка.
    
    Возвращает:
    str: Строка без русских символов.
    """
    # Регулярное выражение для поиска русских символов
    russian_pattern = r'[а-яА-Я]'
    
    # Удаляем русские символы из строки
    cleaned_text = re.sub(russian_pattern, '', text)
    
    return cleaned_text

In [ ]:
files = os.listdir(PATH_TO_IMGS) # Создаём список файлов из тестовой выборки
text_result = [] # Результат обнаружения текста
boxes_result = [] # Результаты обнаружения боксов

# Проходимся по всем файлам из тестовой выборки
for file in files:
    img = cv2.imread(f"{PATH_TO_IMGS}/{file}") # Загрузка изображения

    start = time.time() # Засекаем время начала обработки изображения
    text, img_bbox, bbox = detect_and_recognition(img, model, model_circle, ocr_en) # Обнаружение текста и боксов, и расапознавание текста

    # Подготовка результата распознавания текста к сохранению в submissions.csv
    #text = np.array(text)

    if len(text) == 0:
        text_result.append("\"AM116.06.00.910-01\"")
    else:

        best_match = find_in_BD(text, df)
        article = best_match.iloc[0]
        article = article[1:-1]
        article = article.split(' ')[0]
        serial = best_match.iloc[1]

        is_serial = False
        pattern = r"^.{3}/\d$"
        matches = [s[0] for s in text if re.match(pattern, s[0])]
        if len(matches) != 0:
            is_serial = True
            serial = matches[0]
        
        if is_serial:
            text_str = f"\"{article} {serial}\""
        elif int(serial) == 0:
            text_str = f"\"{article}\""
        else: 
            text_str = f"\"{article} {int(serial)}\""
        text_result.append(text_str)

    # Подготовка результата распознавания боксов к сохранению в submissions.csv
    bboxes = []
    bbox_str = ""
    if len(bbox[0].boxes.xywhn) == 0:
         boxes_result.append("0 0.5 0.5 0.5 0.5\n")
    else:
        for box in bbox:
            bbox_np = (np.array(box.boxes.xywhn))
            for i in range(len(bbox_np)):
                bbox_str += "0 "
                for j in range(len(bbox_np[i])):
                    bbox_str += str(bbox_np[i, j]) + " "
                bbox_str = bbox_str[:-1]
                bbox_str += "\n"
        bbox_str = bbox_str[:-1]
        boxes_result.append(f"{bbox_str}")

    end = time.time()

    # Визуализация результата с номером изображения, распознанным текстом и боксами
    annotation = f" Название изображения: {file} \n Распознанный текст: {text} \n Время на обработку данного изображения: {end - start} секунд"
    fig, ax = plt.subplots(nrows=1, ncols=2)
    ax[0].imshow(img_bbox)
    ax[0].axis("off")
    ax[1].invert_yaxis()
    ax[1].text(0, 0.5, annotation, horizontalalignment='left', verticalalignment='center')
    ax[1].axis("off")

In [ ]:
import pandas as pd

# Сохранение результатов в submissions.csv
data = {
    'image_file': files,
    'label': boxes_result,
    'label_text': text_result
}


In [ ]:
df = pd.DataFrame(data)
df.to_csv('submissions.csv', index=False, quoting=csv.QUOTE_MINIMAL, escapechar=" ")

In [ ]:
# Построчный вывод submissions.csv
with open('submissions.csv', newline='\n') as csvfile:
    spamreader = csv.reader(csvfile)
    for row in spamreader:
        print(', '.join(row))
        print("\n")